In [ ]:
import pandas as pd

data_dir = "/home/davina/Private/dialysis-data"
static_features = [
    "Allergies_19-000093_10082020.txt",
    "Patient_Demographics_19-000093_10082020.txt",
    "Social_History_19-000093_10082020.txt",
]
encounters = [
    "enc_19-000093_10082020.txt",
    "Encounter_Diagnoses_19-000093_10082020.txt",
    "Encounters_19-000093_10082020.txt",
    "Family_History_19-000093_10082020.txt",
    "Flowsheet_Vitals_19-000093_10082020.txt",
    "Hospital_Unit_Transfers_19-000093_10082020.txt",
#     "Labs_19-000093_10082020.txt",
    "Medications_19-000093_10082020.txt",
    "problem_list_diagnoses_19-000093_10082020.txt",
    "Problem_Lists_19-000093_10082020.txt",
#     "Procedures_19-000093_10082020.txt",
]
provider_mapping_file = "providers_19-000093_10082020.txt"
outcome_file = "CRRT Deidentified 2017-2019.csv"

In [ ]:
from functools import reduce
from typing import List

def read_files_and_combine(files: List[str]):
    dfs = []

    for file in files:
        try:
            dfs.append(pd.read_csv(f"{data_dir}/{file}"))
        except:
            print(f"Unexpected encoding in {file}")
            default_guess = "cp1252"
            import os
            # get file encoding using file -i and extracting name with sed
            # ref: https://unix.stackexchange.com/a/393949
            # -n: don't print unless we say. s/ search, .* match any, charset=, // remove text up until after =, print remaining
            command = f"file -i {data_dir}/{file} | sed -n 's/.*charset=//p'"
            # [:-1] ignore newline
            encoding = os.popen(command).read()[:-1]
            print(f"Encoding was {encoding} instead of assumed utf-8.")
            if encoding == "unknown-8bit":
                print(f"Assuming {default_guess}...")
                dfs.append(pd.read_csv(f"{data_dir}/{file}",  encoding=default_guess))
            else:
                dfs.append(pd.read_csv(f"{data_dir}/{file}",  encoding=encoding))
    combined = reduce(lambda df1, df2: pd.merge(df1, df2, on="IP_PATIENT_ID", how="inner"), dfs)
    return combined

# Preprocessing features

In [ ]:
static_df = read_files_and_combine(static_features)

In [ ]:
# map provider id to type
provider_mapping = pd.read_csv(f"{data_dir}/{provider_mapping_file}")
provider_mapping = dict(zip(provider_mapping["IP_PROVIDER_ID"], provider_mapping["PROVIDER_TYPE"]))
static_df["PCP_IP_PROVIDER_ID"] = static_df["PCP_IP_PROVIDER_ID"].map(provider_mapping)
static_df.rename(columns={"PCP_IP_PROVIDER_ID" : "PCP_PROVIDER_TYPE"}, inplace=True)

In [ ]:
static_df

In [ ]:
static_df["IP_PATIENT_ID"].nunique()

## LONGITUDINAL FEATURES

- encounters: basic info about each encounter
- diagnoses: combine all diagnoses for a given patient across all encounters then one hot encode for the top 10 most common diagnoses
- history: ??
- flowsheet vitals: aggregatie across each type of vital sign (min, max, mean, etc)
- hospital unit transfers: i think we shouldn't use this for now
- labs: aggregate across each continuous lab (min, max, etc).
    - maybe look at all the labs that are not continuous values and decide what to do with those
- medications: treat like diagnoses
- procedures: treat like diagnoses
    - use code or name, just keep mapping intact
- how is problem_list_diagnoses different from problem_lists and dignoses?

We're going to have to do feature selection.

In [ ]:
import numpy as np
from scipy.stats import skew

aggregate_functions = [min, max, np.mean, np.std, skew, len]
longitudinal_df = []

### Diagnoses
How do we want to deal with icd9 vs icd10?

In [ ]:
dx_df = read_files_and_combine(["Encounter_Diagnoses_19-000093_10082020.txt"])
# Top N codes
top_n = 15
dx_df[dx_df["ICD_TYPE"] == 10].groupby("ICD_CODE").size().sort_values(ascending=False)[:top_n]

#### ICD9 VS ICD10

In [ ]:
icd10_n = (dx_df['ICD_TYPE'] == 10).sum()
icd9_n = (dx_df['ICD_TYPE'] == 9).sum()
f"ICD 10: {icd10_n}, ICD 9: {icd9_n}, ratio of 10:9: {icd10_n / icd9_n}"

In [ ]:
# Date ranges for ICD9 codes (min and max)
icd9_dx_dates = pd.to_datetime(dx_df[dx_df["ICD_TYPE"] == 9]["DIAGNOSIS_DATE"])
f"min date: {icd9_dx_dates.min()}, max_date: {icd9_dx_dates.max()}"

ICD9 is 2013 - 2015, outcomes are 2018-2019, so we will ignore all ICD9 codes and use HCUP to map ICD10 codes to CSSR categories to reduce the number of categories.

#### ICD10 to CSSR

In [ ]:
from hcuppy.ccs import CCSEngine
ce = CCSEngine(mode="dx")
# convert icd10 to ccs
dx_df["CCS_CODE"] = dx_df[dx_df["ICD_TYPE"] == 10]["ICD_CODE"].apply(lambda icd_code: ce.get_ccs(icd_code)["ccs"])
dx_df.groupby("CCS_CODE").size()

### Vitals

These looks a little strange right now.

In [ ]:
vitals_df = read_files_and_combine(["Flowsheet_Vitals_19-000093_10082020.txt"])

In [ ]:
old_size = vitals_df.shape[0]

# drop duplicates for the same patient for the same vital (taken at same time indicates duplicate)
vitals_df = vitals_df.drop_duplicates(subset=["IP_PATIENT_ID", "VITAL_SIGN_TYPE", "VITAL_SIGN_TAKEN_TIME"])
f"Dropped {old_size - vitals_df.shape[0]} rows that were duplicates."

In [ ]:
# Split BP into SBP and DBP
vitals_df["VITAL_SIGN_TYPE"].replace({"BP": "SBP/DBP"}, inplace=True)
explode_cols = ["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"]
def try_split_col(col: pd.Series):
    # Split col with "/" in it (only BP values and name) from explode_cols
    try:
        return col.str.split("/").explode()
    except:
        return col

# Ref: https://stackoverflow.com/a/57122617/1888794
# don't explode the columsn you set index to, explode the rest via apply, reset everything to normal
vitals_df = (vitals_df.set_index(list(vitals_df.columns.difference(explode_cols)))
                    .apply(try_split_col)
                    .reset_index()
                    .reindex(vitals_df.columns, axis=1))

In [ ]:
# these vitals are not float point numbers, we want to ignore them and then convert the vitals to float to aggregate
ignore_vitals = ["O2 Device"]
ignore_mask = ~vitals_df["VITAL_SIGN_TYPE"].isin(ignore_vitals)
vitals_df = vitals_df[ignore_mask]
# convert to float
vitals_df["VITAL_SIGN_VALUE"] = vitals_df["VITAL_SIGN_VALUE"].astype(float)
# Aggregate
vitals_df.groupby(["IP_PATIENT_ID", "VITAL_SIGN_TYPE"]).agg({"VITAL_SIGN_VALUE": aggregate_functions})

### Medications

In [ ]:
rx_df = read_files_and_combine(["Medications_19-000093_10082020.txt"])


In [ ]:
rx_df.groupby("GENERIC_NAME").size().sort_values(ascending=False)

### Labs: TODO

In [ ]:
labs_df = read_files_and_combine(["Labs_19-000093_10082020.txt"])

In [ ]:

labs_df.groupby(["IP_PATIENT_ID", "PROC_ID"]).agg({"RESULTS": aggregate_functions})

### Problems

In [ ]:
problems_df = read_files_and_combine(["Problem_Lists_19-000093_10082020.txt"])

In [ ]:
problems_df.groupby("problem_status").size()

### Procedures
TODO: We need to know if ICD9, ICD10, or APT Code?
We can otherwise use hcuppy to convert procedures: `hcuppy.prcls` procedure class for those that work otherwise ignore or drop.

In [ ]:
procedure_df = read_files_and_combine(["Procedures_19-000093_10082020.txt"])

In [ ]:
procedure_df.groupby("PROC_CODE").size().sort_values(ascending=False)

# Load + Preproc Outcomes

In [ ]:
# get first sheet only
outcomes = pd.read_csv(f"{data_dir}/{outcome_file}")

# Exclude pediatric data
exclude_peds = outcomes["Hospital name"] != "UCLA MEDICAL CENTER- PEDIATRICS"
outcomes = outcomes[exclude_peds]

# TODO: include CRRT Total Days as predictive feature

outcomes

In [ ]:
# Date range for outcomes
outcome_end_date = pd.to_datetime(outcomes["End Date"])
f"min date: {outcome_end_date.min()}. max date: {outcome_end_date.max()}"

## Validate Outcomes

In [ ]:
positive_outcomes = ["Recov. renal funct.", "Transitioned to HD"]
negative_outcomes = ["Palliative Care", "Expired "] 
outcome_cols = positive_outcomes + negative_outcomes
outcomes[outcome_cols]

In [ ]:
# Each row should have exactly 1 1.0 value (one-hot of the 4 cols)
bad_rows = outcomes[outcome_cols].fillna(0).sum(axis=1) == 0
outcomes[bad_rows]
## TODO: Should i drop the bad row?

## Construct outcome feature (recommend dialysis)

In [ ]:
recommend_dialysis = (outcomes["Recov. renal funct."] == 1) | (outcomes["Transitioned to HD"] == 1)
outcomes["recommend_dialysis"] =  recommend_dialysis.astype(int)

# To combine with features
outcome_df = outcomes[["IP_PATIENT_ID", "recommend_dialysis"]]

In [ ]:
sum(outcome_df["recommend_dialysis"])/len(outcome_df) * 100

# Merge features with outcome

In [ ]:
# features_with_outcomes = pd.merge(combined, outcome_df, on="IP_PATIENT_ID", how="inner")
features_with_outcomes = reduce(lambda df1, df2: pd.merge(df1, df2, on="IP_PATIENT_ID", how="inner"),
                                [static_df, longitudinal_df, outcome_df])
features_with_outcomes